# Particle statistics calculations

The git repository for this notebook contains a includes a standard development environment that downloads the necessary dataset and install all required packages. If using VS Code you can use the _Dev containers: Reopen in container_ command to run this notebook locally within a tested environment.

In [ ]:
import gzip
import math
import pickle
import xml.etree.ElementTree as ET
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import seaborn as sns
from ipywidgets import Button, interact, interactive
from scipy import stats
from scipy.stats import bootstrap
from tqdm.auto import tqdm
from transformers import pipeline

DATA_HOME = Path("data") / "LZ-05"

sns.set(
    context="notebook",
    style="ticks",
    font="Arial",
    font_scale=1.1,
    rc={"svg.fonttype": "none", "lines.linewidth": 1.6, "figure.autolayout": True},
)

In [ ]:
excel_file = pd.read_excel("Experiments.xlsx").query('Active == "Yes"')
excel_file

In [ ]:
sample_image_files = {
    filename: DATA_HOME / filename
    for filename in excel_file["Filename"]
    if (DATA_HOME / filename).exists()
}

len(sample_image_files)

In [ ]:
# read image files
imgs_RGB = {id: PIL.Image.open(path) for id, path in tqdm(sample_image_files.items())}

# shrink by 1/4
imgs_small_RGB = {
    id: img.resize((img.width // 4, img.height // 4), resample=PIL.Image.BILINEAR)
    for id, img in tqdm(imgs_RGB.items())
}

Show the first few images

In [ ]:
def show_image(id):
    data = imgs_small_RGB[id]
    plt.imshow(data)
    plt.title(id)


interact(show_image, id=sample_image_files.keys());

## Generate or load masks for all images

In [ ]:
mask_generator = pipeline(
    task="mask-generation",
    model="facebook/sam-vit-base",
    device="cuda",
    points_per_crop=64,
    pred_iou_thresh=0.2,
    stability_score_thresh=0.2,
    crops_nms_thresh=0.1,
    points_per_batch=128,
)

In [ ]:
mask_file = Path("out/masks.pkl.gz")
mask_file.parent.mkdir(parents=True, exist_ok=True)

if mask_file.exists():
    with gzip.open(mask_file, "rb") as f:
        masks = {
            id: mask for id, mask in pickle.load(f).items() if id in imgs_small_RGB
        }
else:
    masks = {}

new_masks = {
    id: np.array(mask_generator(img)["masks"])
    for id, img in tqdm(imgs_small_RGB.items())
    if id not in masks
}
# Save masks to pickle file
if new_masks:
    masks.update(new_masks)
    if mask_file.exists():
        # make a backup of current mask_file with a timestamp
        backup_file = mask_file.with_name(
            f'{mask_file.stem}_{pd.Timestamp.now().strftime("%Y%m%d%H%M%S")}{mask_file.suffix}'
        )
        mask_file.rename(backup_file)
    with gzip.open(mask_file, "wb") as f:
        pickle.dump(masks, f)
{id: len(img_masks) for id, img_masks in masks.items()}

Mask area historgram

In [ ]:
def show_hist(id):
    areas = np.log10([sample_mask.sum() for sample_mask in masks[id]])
    plt.hist(areas, bins=100)
    plt.title(id)
    plt.xlabel("log10(area/px^2)")


interact(show_hist, id=sample_image_files.keys())

### Remove masks that are too big or too small


In [ ]:
MAX_MASK_AREA = 1500
MIN_MASK_AREA = 50
masks_filtered = {
    id: np.array(
        [mask for mask in img_masks if MIN_MASK_AREA < mask.sum() < MAX_MASK_AREA]
    )
    for id, img_masks in masks.items()
}
{id: len(img_masks) for id, img_masks in masks_filtered.items()}

## Visualising the location of masks for each image

In [ ]:
mask_tensors = {
    img_id: masks_filtered[img_id].astype(np.uint8) for img_id in tqdm(masks_filtered)
}

In [ ]:
interact(
    lambda sample_name: plt.imshow(mask_tensors[sample_name].sum(axis=0))
    and plt.title(sample_name)
    and None,
    sample_name=mask_tensors.keys(),
)

In [ ]:
all_included = {
    img_id: np.sum(
        mask_tensors[img_id].astype(int)
        * np.random.randint(1, 16, size=(mask_tensors[img_id].shape[0], 1, 1)),
        axis=0,
    )
    for img_id in tqdm(mask_tensors)
}

In [ ]:
# Color map where 0 is black and 1-16 are different colors
cmap = matplotlib.colormaps["tab20"]
# set 0 to black
cmap.colors = ((0, 0, 0, 1),) + cmap.colors[1:]


def show_fn(img_id):
    filename = img_id.replace(".jpg", ".svg")
    img = imgs_small_RGB[img_id]
    mask = all_included[img_id]
    f, (img_ax, mask_ax) = plt.subplots(1, 2, figsize=(12, 6))
    img_ax.imshow(img)
    mask_ax.imshow(mask, cmap=cmap, interpolation="none")
    mask_ax.set_title(f"{len(masks_filtered[img_id])} masks")

    b = Button(description=f"Save to {filename}")
    b.on_click(lambda x: f.savefig(filename, format="svg", transparent=True))
    display(b)


interactive(show_fn, img_id=all_included.keys())

### Add mask count and area to the dataframe

In [ ]:
mask_areas = [
    pd.DataFrame(
        {
            "mask_area": [mask.sum() for mask in masks],
            "Filename": img_id,
        },
    )
    for img_id, masks in masks_filtered.items()
]
count_df = excel_file.merge(
    pd.DataFrame(
        {"# particles": [len(masks) for masks in masks_filtered.values()]},
        index=masks_filtered.keys(),
    ),
    left_on="Filename",
    right_index=True,
)
mask_df = excel_file.merge(pd.concat(mask_areas), on="Filename")
mask_df = mask_df.assign(
    **{"Diameter (µm)": np.sqrt(mask_df["mask_area"] / np.pi) * 2 / 0.906}
)
mask_df

In [ ]:
count_df

In [ ]:
sns.lineplot(
    data=count_df.query("`Number of pulses` < 2400").rename(
        columns={"Number of pulses": "# pulses"}
    ),
    x="# pulses",
    y="# particles",
    hue="Pulse duration",
)
plt.savefig(
    "out/num_particles vs num_pulses.svg",
    transparent=True,
    bbox_inches="tight",
    pad_inches=0.1,
)

In [ ]:
sns.boxplot(
    data=count_df.query("`Number of pulses` < 2400").rename(
        columns={"Number of pulses": "# pulses"}
    ),
    x="# pulses",
    y="# particles",
    hue="Pulse duration",
    native_scale=True,
    fliersize=1,
    showfliers=False,
    # width=0.4,
    # log_scale=True
)

In [ ]:
sns.boxplot(
    data=mask_df.query("`Number of pulses` < 2400").rename(
        columns={"Number of pulses": "# pulses"}
    ),
    x="# pulses",
    y="Diameter (µm)",
    hue="Pulse duration",
    native_scale=True,
    fliersize=1,
    showfliers=False,
    # width=0.4,
    # log_scale=True
)
plt.savefig(
    "out/diameter vs num_pulses.svg",
    transparent=True,
    bbox_inches="tight",
    pad_inches=0.1,
)

In [ ]:
sns.lineplot(
    data=mask_df.query("`Number of pulses` < 2400").rename(
        columns={"Number of pulses": "# pulses"}
    ),
    x="# pulses",
    y="Diameter (µm)",
    hue="Pulse duration",
    # width=0.4,
    # log_scale=True
)

In [ ]:
count_df.to_excel("out/mask_counts.xlsx", index=False)
mask_df.to_excel("out/mask_areas.xlsx", index=False)

In [ ]:
def parse_svg(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Find the scale bar
    scale_value = 200.0
    scale_line = root.find(".//*[@id='path9']")
    scale_width = float(scale_line.attrib["d"].split()[-1])

    # Calculate the conversion factor
    conversion_factor = scale_value / scale_width

    # Find all ellipse and circle elements
    particles = root.findall(".//*[@cx][@cy]")

    data = []
    for particle in particles:
        if "rx" in particle.attrib and "ry" in particle.attrib:
            rx = float(particle.attrib["rx"])
            ry = float(particle.attrib["ry"])
        elif "r" in particle.attrib:
            rx = ry = float(particle.attrib["r"])
        else:
            continue

        diameter = math.sqrt(rx * ry) * 2 * conversion_factor
        data.append(diameter)

    return pd.DataFrame(data, columns=["Diameter (µm)"])


# Run the analysis on the SVG file
aerosol_aerosol_df = parse_svg("Figure 8 - sizing.svg")


def compare_groups(group1, group2, group1_name, group2_name):
    combined_data = pd.concat(
        [group1.assign(Group=group1_name), group2.assign(Group=group2_name)]
    )

    plt.figure(figsize=(12, 6))

    plt.subplot(121)
    sns.histplot(
        data=combined_data, x="Diameter (µm)", hue="Group", kde=True, element="step"
    )
    plt.title("Histogram of Particle Diameters")

    plt.subplot(122)
    sns.boxplot(data=combined_data, x="Group", y="Diameter (µm)")
    plt.title("Box Plot of Particle Diameters")

    plt.tight_layout()
    plt.savefig("particle_diameter_comparison.png")
    print("\nVisualization saved as 'particle_diameter_comparison.png'")


# Perform the comparison
compare_groups(
    aerosol_aerosol_df,
    mask_df,
    group1_name="Aerosol–aerosol",
    group2_name="Aerosol-bulk",
)

In [ ]:
def compare_groups(
    group1, group2, group1_name="Aerosol–aerosol", group2_name="Aerosol-bulk"
):
    combined_data = pd.concat(
        [group1.assign(Group=group1_name), group2.assign(Group=group2_name)]
    )

    desc_stats = combined_data.groupby("Group")["Diameter (µm)"].describe()
    print("Descriptive Statistics:")
    print(desc_stats)
    print(f"\nSample sizes: {group1_name}: {len(group1)}, {group2_name}: {len(group2)}")
    print("\n")

    t_stat, p_value_t = stats.ttest_ind(
        group1["Diameter (µm)"], group2["Diameter (µm)"], equal_var=False
    )

    print(f"t-statistic = {t_stat:.4f}, p-value = {p_value_t:.4f}")

    def diff_in_means(data1, data2):
        return np.mean(data1) - np.mean(data2)

    bootstrap_result = bootstrap(
        (group1["Diameter (µm)"], group2["Diameter (µm)"]),
        diff_in_means,
        n_resamples=10000,
    )
    ci_low, ci_high = bootstrap_result.confidence_interval
    print(f"\nBootstrap 95% CI for difference in means: ({ci_low:.4f}, {ci_high:.4f})")

    plt.figure(figsize=(12, 6))

    plt.subplot(121)
    sns.kdeplot(data=combined_data, x="Diameter (µm)", hue="Group", common_norm=False)
    plt.title("Density Plot of Particle Diameters")

    plt.subplot(122)
    sns.boxplot(data=combined_data, x="Group", y="Diameter (µm)")
    plt.title("Box Plot of Particle Diameters")

    plt.tight_layout()
    plt.savefig(
        "particle_diameter_comparison.png",
        transparent=True,
        bbox_inches="tight",
        dpi=200,
    )


compare_groups(aerosol_aerosol_df, mask_df)

## Pairwise t-tests between different conditions (pulse length, pulse count)

In [ ]:
def pairwise_ttest_comparison(df):
    # Get unique combinations of 'Number of pulses' and 'Pulse duration'
    pulse_combinations = df.groupby(["Number of pulses", "Pulse duration"])

    # Create a list of all unique pairs of combinations
    combination_pairs = list(combinations(pulse_combinations.groups.keys(), 2))

    # Initialize lists to store results
    pair1_list, pair2_list, tstat_list, pvalue_list = [], [], [], []

    for pair in combination_pairs:
        group1 = pulse_combinations.get_group(pair[0])["Diameter (µm)"]
        group2 = pulse_combinations.get_group(pair[1])["Diameter (µm)"]

        # Perform Welch's t-test
        t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)

        # Store results
        pair1_list.append(f"{pair[0][0]}, {pair[0][1]}")
        pair2_list.append(f"{pair[1][0]}, {pair[1][1]}")
        tstat_list.append(t_stat)
        pvalue_list.append(p_value)

    # Create a dataframe with the results
    results_df = pd.DataFrame(
        {
            "Combination 1": pair1_list,
            "Combination 2": pair2_list,
            "t-statistic": tstat_list,
            "p-value": pvalue_list,
        }
    )

    return results_df


# Assuming mask_df is already loaded and has 'Number of pulses', 'Pulse duration', and 'Diameter (µm)' columns
results = pairwise_ttest_comparison(mask_df)

results.to_excel("out/pairwise_ttest_results.xlsx", index=False)
results

## Pairwise difference of means bootstrap confidence intervals

In [ ]:
def bootstrap_difference(group1, group2, n_resamples=10000):
    return bootstrap(
        (group1, group2), lambda x, y: np.mean(x) - np.mean(y), n_resamples=n_resamples
    ).confidence_interval


def create_comparison_matrix(df):
    combinations = df.groupby(["Number of pulses", "Pulse duration"])
    combo_list = list(combinations.groups.keys())
    multi_index = pd.MultiIndex.from_tuples(
        combo_list, names=["Number of pulses", "Pulse duration"]
    )

    result_df = pd.DataFrame(index=multi_index, columns=multi_index)

    for combo1 in combo_list:
        group1 = combinations.get_group(combo1)["Diameter (µm)"]
        for combo2 in combo_list:
            group2 = combinations.get_group(combo2)["Diameter (µm)"]
            ci_low, ci_high = bootstrap_difference(group1, group2)
            result_df.loc[combo1, combo2] = f"{ci_low:.2f} to {ci_high:.2f}"

    return result_df


def plot_heatmap(result_matrix):
    plt.figure(figsize=(12, 10))
    heatmap_data = result_matrix.applymap(
        lambda x: float(x.split()[0]) if x != "0" else 0
    )
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="RdYlBu_r", center=0)
    plt.tight_layout()
    plt.savefig("out/pulse_comparison_heatmap.png", dpi=200, transparent=True)
    plt.close()


result_matrix = create_comparison_matrix(mask_df)
plot_heatmap(result_matrix)

In [77]:
result_matrix.to_excel("out/pulse_comparison_matrix.xlsx")